In [1]:
%load_ext lab_black

In [2]:
import re
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from sklearn import utils
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import ensemble
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from matplotlib import pyplot as plt

In [232]:
df_all = pd.read_csv("data_basic.csv")

In [277]:
df_additional = pd.read_csv("additional_info_new.csv")

In [234]:
df_keywords = pd.read_csv("keywords_processed.csv")

In [278]:
df_all = df_all.merge(
    df_additional, how="left", left_on="originalTitle", right_on="title"
)

In [279]:
df_all.drop(columns="title", inplace=True)

In [282]:
list_of_titles = df_all.originalTitle
df = df_all.iloc[:, 3:]

In [283]:
df.drop(columns="numVotes", inplace=True)

In [7]:
df = df.replace("no value", np.nan)

In [8]:
df.runtimeMinutes = pd.to_numeric(df.runtimeMinutes)

In [286]:
df["runtimeMinutes"].fillna((df["runtimeMinutes"].mean()), inplace=True)

In [287]:
df[["startYear", "runtimeMinutes", "averageRating"]] = df[
    ["startYear", "runtimeMinutes", "averageRating"]
].astype("float32")

In [4]:
df = pd.read_csv("data/df_sample_to_check.csv")

C:\Users\Alicja Kocieniewska\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df.runtimeMinutes = (df.runtimeMinutes - df.runtimeMinutes.min()) / (
    df.runtimeMinutes.max() - df.runtimeMinutes.min()
)

In [289]:
df["howOld"] = 2020 - df["startYear"]
df.drop(columns="startYear", inplace=True)

In [10]:
df["howOld"] = (df["howOld"] - df["howOld"].min()) / (
    df["howOld"].max() - df["howOld"].min()
)

In [297]:
df.iloc[:, 2:-2] = df.iloc[:, 2:-2].astype("bool")

In [305]:
df["is_black_and_white"].fillna(False, inplace=True)

In [16]:
df.drop(columns=["Unnamed: 0", "tconst", "originalTitle", "title"], inplace=True)

In [21]:
# x i y
X = df.drop(columns="averageRating")
y = df.averageRating

Exception ignored in: <function DMatrix.__del__ at 0x0000028CBA84D840>
Traceback (most recent call last):
  File "C:\Users\Alicja Kocieniewska\Anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Fit regression model
params = {
    "n_estimators": 5000,
    "max_depth": 20,
    "learning_rate": 0.05,
    "gamma": 20,
    "loss": "ls",
}
model = xgb.XGBRegressor(**params)

model.fit(X_train, y_train)
mse = mean_squared_error(y_test, np.round(model.predict(X_test), 1))
print(mse)
r2 = r2_score(y_test, np.round(model.predict(X_test), 1))
mae = mean_absolute_error(y_test, np.round(model.predict(X_test), 1))
print(mae)

In [ ]:
mae = mean_absolute_error(y_train, np.round(model.predict(X_train), 1))
print(mae)

In [ ]:
plt.figure(figsize=(30, 30))
xgb.plot_importance(model, max_num_features=40)
plt.show()

In [ ]:
print(r2)

In [ ]:
np.round(model.predict(X_test), 1)[:10]

In [ ]:
y_test[:10]